# Necessary Imports and Settings

In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import transformers
import torch
import os
import nltk
import pandas as pd
import torch
import numpy as np
from jinja2 import Template
import xmltodict
import pickle
from collections import defaultdict
from fuzzywuzzy import fuzz
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import time
import copy

import sys
sys.path.append('/scratch/users/bozyurt20/hpc_run/utilities')
sys.path.append("/scratch/users/bozyurt20/hpc_run/blobs/")
from util_research import *

#from toy_dataset import contexts

max_len = 512
num_layers = 24
d_model = 4096

tokenizer = AutoTokenizer.from_pretrained("bigscience/T0pp", truncation_side="right", add_prefix_space=True)

In [97]:
model = AutoModelForSeq2SeqLM.from_pretrained("bigscience/T0pp", device_map="balanced", load_in_8bit=True)

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /kuacc/users/bozyurt20/.conda/envs/hf/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 110
CUDA SETUP: Loading binary /kuacc/users/bozyurt20/.conda/envs/hf/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda110.so...


In [3]:
names_1 = ["John", "Harry", "Andrew"]
names_2 = ["Henry", "David", "Sophia"]
names_3 = ["Olivia", "Emma", "Lisa"]

cities_1 = ["London", "Paris", "Oslo"]
cities_2 = ["Sydney", "Cairo", "Seoul"]
cities_3 = ["Istanbul", "Beijing", "Rome"]

class DataSample_ThreeSentences():
    def __init__(self, name_1, name_2, name_3, city_1, city_2, city_3):
        self.name_1 = name_1
        self.name_2 = name_2
        self.name_3 = name_3
        self.city_1 = city_1
        self.city_2 = city_2
        self.city_3 = city_3
        self.prev_context = name_1 + " travelled to " + city_1 + ". " + name_2 + " travelled to " + city_2 + ". " + name_3 + " travelled to " + city_3 + ". Where did " + name_3 + " travel to?" 
        self.current_context_1 = name_1 + " met Lucas. Lucas was 30 years old. Where is " + name_1 + "?"
        self.current_context_2 = name_2 + " met Lucas. Lucas was 30 years old. Where is " + name_2 + "?"
        self.current_context_3 = name_3 + " met Lucas. Lucas was 30 years old. Where is " + name_3 + "?"
    
    def add_prev_encoding(self, encoding):
        self.prev_encoding = encoding
    def add_current_encoding_1(self, encoding):
        self.curr_encoding_1 = encoding
    def add_current_encoding_2(self, encoding):
        self.curr_encoding_2 = encoding
    def add_current_encoding_3(self, encoding):
        self.curr_encoding_3 = encoding
    def add_generation(self, model_out):
        self.model_out = model_out
        
class ThreeSentenceRewriteResultCase1():
    def __init__ (self, context_1, context_2, generation_1, generation_2, generation_rewritten, answer_1, answer_2):
        
        #self.data_point_1 = data_point_1
        #self.data_point_2 = data_point_2
        self.context_1 = context_1
        self.context_2 = context_2
        self.generation_1 = generation_1
        self.generation_2 = generation_2
        self.generation_rewritten = generation_rewritten
        self.answer_1 = answer_1
        self.answer_2 = answer_2
        
class ThreeSentenceRewriteResultCase2():
    def __init__ (self, context_1, context_2, generation_1, generation_2, generation_rewritten, answer_1, answer_2):
        
        #self.data_point_1 = data_point_1
        #self.data_point_2 = data_point_2
        self.context_1 = context_1
        self.context_2 = context_2
        self.generation_1 = generation_1
        self.generation_2 = generation_2
        self.generation_rewritten = generation_rewritten
        self.answer_1 = answer_1
        self.answer_2 = answer_2

In [98]:
def case1_rewrite(pairs_to_observe, ids_to_move):
    
    case1_results = []

    for sample_1, sample_2 in pairs_to_observe:

        encoding_sample_1 = sample_1.prev_encoding

        context_1 = sample_1.prev_context
        context_2 = sample_2.prev_context
        generation_1 = sample_1.model_out
        generation_2 = sample_2.model_out
        answer_1 = sample_1.city_3
        answer_2 = sample_2.city_3

        special_hidden = encoding_sample_1.special_hidden_states # 24 x (1, T, d)
        len_input_ids = special_hidden[0].shape[1]
        special_reformatted = torch.zeros(num_layers, len_input_ids, d_model) # (24, T, d)
        for i, hidden in enumerate(special_hidden):
            special_reformatted[i:i+1, :, :] = hidden

        entities_hidden_states = special_reformatted[:, ids_to_move[0], :].unsqueeze(0) # 1, 24, d
        
        for tok_id in ids_to_move[1:]:
            entity_hidden_states = special_reformatted[:, tok_id, :].unsqueeze(0) # 1, 24, d
            entities_hidden_states = torch.cat((entities_hidden_states,
                                                entity_hidden_states), dim=0)

        input_ids = tokenizer.encode(context_2, return_tensors="pt").to(model.encoder.device)
        generation_rewritten = model.generate(input_ids=input_ids, 
                                             max_new_tokens=10,
                                             entity_hidden_states=entities_hidden_states,
                                             entity_inds=ids_to_move,
                                             return_dict_in_generate=True, 
                                             output_scores=True)

        case1_results.append(ThreeSentenceRewriteResultCase1(context_1, context_2, generation_1, generation_2, generation_rewritten, answer_1, answer_2))
        
    return case1_results

In [99]:
def case2_rewrite(pairs_to_observe, ids_to_move):

    case2_results = []

    for sample_1, sample_2 in pairs_to_observe:

        encoding_sample_1 = sample_1.prev_encoding
        encoding_sample_2 = sample_2.prev_encoding

        context_1 = sample_1.prev_context
        context_2 = sample_2.prev_context
        generation_1 = sample_1.model_out
        generation_2 = sample_2.model_out
        answer_1 = sample_1.city_3
        answer_2 = sample_2.city_3

        encoding_sample_2_copy = copy.deepcopy(encoding_sample_2)

        for i in ids_to_move:
            encoding_sample_2_copy.last_hidden_state[:,i:i+1,:] = encoding_sample_1.last_hidden_state[:,i:i+1,:]
        
        generation_rewritten = model.generate(encoder_outputs=encoding_sample_2_copy, 
                                               max_new_tokens=10,
                                               return_dict_in_generate=True, 
                                               output_scores=True)
        case2_results.append(ThreeSentenceRewriteResultCase2(context_1, context_2, generation_1, generation_2, generation_rewritten, answer_1, answer_2))
        
    return case2_results

# Case 1 Pre-Encodings

In [5]:
data_points_three_sentences_case1 = []
for name_1 in names_1:
    for name_2 in names_2:
        for name_3 in names_3:
            for city_1 in cities_1:
                for city_2 in cities_2:
                    for city_3 in cities_3:
                        sample = DataSample_ThreeSentences(name_1, name_2, name_3, city_1, city_2, city_3)
                        
                        prev_context = sample.prev_context
                        
                        prev_tokens = tokenizer.encode(prev_context, return_tensors="pt").to(model.encoder.device)
                        with torch.no_grad():
                            encoded_prev = model.encoder(prev_tokens, output_special=True, output_hidden_states=True)
                        with torch.no_grad():
                            model_out = model.generate(input_ids=prev_tokens, 
                                         max_new_tokens=10,
                                         return_dict_in_generate=True, 
                                         output_scores=True)

                        sample.add_prev_encoding(encoded_prev)
                        sample.add_generation(model_out)
                        
                        data_points_three_sentences_case1.append(sample)

In [6]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Rewrite/ThreeSentenceDataEncodings/data_points_three_sentences_case1.txt", "wb") as f:
    pickle.dump(data_points_three_sentences_case1, f)

In [4]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Rewrite/ThreeSentenceDataEncodings/data_points_three_sentences_case1.txt", "rb") as f:
    data_points_three_sentences_case1 = pickle.load(f)

# Case 2 Pre-Encodings

In [10]:
data_points_three_sentences_case2 = []
for sample in data_points_three_sentences_case1:
    name_1 = sample.name_1
    name_2 = sample.name_2
    name_3 = sample.name_3
    city_1 = sample.city_1
    city_2 = sample.city_2
    city_3 = sample.city_3
    prev_context = sample.prev_context 
    current_context_1 = sample.current_context_1
    current_context_2 = sample.current_context_2
    current_context_3 = sample.current_context_3
    prev_encoding = sample.prev_encoding
    
    sample = DataSample_ThreeSentences(name_1, name_2, name_3, city_1, city_2, city_3)
    
    sample.add_prev_encoding(prev_encoding)
    
    curr_tokens_1 = tokenizer.encode(current_context_1, return_tensors="pt").to(model.encoder.device)
    with torch.no_grad():
        encoded_curr_1 = model.encoder(curr_tokens_1)
    sample.add_current_encoding_1(encoded_curr_1)

    curr_tokens_2 = tokenizer.encode(current_context_2, return_tensors="pt").to(model.encoder.device)
    with torch.no_grad():
        encoded_curr_2 = model.encoder(curr_tokens_2)
    sample.add_current_encoding_2(encoded_curr_2)
    
    curr_tokens_3 = tokenizer.encode(current_context_3, return_tensors="pt").to(model.encoder.device)
    with torch.no_grad():
        encoded_curr_3 = model.encoder(curr_tokens_3)
    sample.add_current_encoding_3(encoded_curr_3)

    data_points_three_sentences_case2.append(sample)
    

In [11]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Rewrite/ThreeSentenceDataEncodings/data_points_three_sentences_case2.txt", "wb") as f:
    pickle.dump(data_points_three_sentences_case2, f)

In [5]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Rewrite/ThreeSentenceDataEncodings/data_points_three_sentences_case2.txt", "rb") as f:
    data_points_three_sentences_case2 = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: '/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Rewrite/ThreeSentenceDataEncodings/data_points_three_sentences_case2.txt'

# Pairs to Rewrite

In [101]:
pairs_to_observe = []

for i in range(len(data_points_three_sentences_case1)-1):
    for j in range(i+1, len(data_points_three_sentences_case1)):
        sample_1 = data_points_three_sentences_case1[i]
        sample_2 = data_points_three_sentences_case1[j]

        if (sample_1.name_1 == sample_2.name_1
            and sample_1.name_2 == sample_2.name_2 
            and sample_1.name_3 == sample_2.name_3
            and sample_1.city_1 == sample_2.city_1 
            and sample_1.city_2 == sample_2.city_2 
            and sample_1.city_3 != sample_2.city_3):

            """print(sample_1.prev_context)
            print(sample_2.prev_context)
            
            print()
            print()"""
            
            pairs_to_observe.append((sample_1, sample_2))


In [8]:
len(data_points_three_sentences_case1)

729

In [9]:
len(pairs_to_observe)

729

# Case 1 Rewrite

In [12]:
ids_to_move = [9] # middle _to token

case1_middle_to_results = case1_rewrite(pairs_to_observe, ids_to_move)

with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Rewrite/Results/case1_middle_to_results.txt", "wb") as f:
    pickle.dump(case1_middle_to_results, f)

In [6]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Rewrite/Results/case1_middle_to_results.txt", "rb") as f:
    case1_middle_to_results = pickle.load(f)

In [103]:
ids_to_move = [16]

case1_loc_results = case1_rewrite(pairs_to_observe, ids_to_move)

with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Rewrite/Results/case1_loc_results.txt", "wb") as f:
    pickle.dump(case1_loc_results, f)

In [104]:
ids_to_move = [12, 13, 14, 15]

case1_allexceptloc_results = case1_rewrite(pairs_to_observe, ids_to_move)

with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Rewrite/Results/case1_allexceptloc_results.txt", "wb") as f:
    pickle.dump(case1_allexceptloc_results, f)

In [105]:
ids_to_move = [12, 13, 14, 15, 16]

case1_alltok_results = case1_rewrite(pairs_to_observe, ids_to_move)

with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Rewrite/Results/case1_alltok_results.txt", "wb") as f:
    pickle.dump(case1_alltok_results, f)

In [118]:
ids_to_move = list(range(16))

case1_all3sentexceptlastloc_results = case1_rewrite(pairs_to_observe, ids_to_move)

with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Rewrite/Results/case1_all3sentexceptlastloc_results.txt", "wb") as f:
    pickle.dump(case1_all3sentexceptlastloc_results, f)

# Case 2 Rewrite

In [21]:
ids_to_move = [9] # middle _to token

case2_middle_to_results = case2_rewrite(pairs_to_observe, ids_to_move)

with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Rewrite/Results/case2_middle_to_results.txt", "wb") as f:
    pickle.dump(case2_middle_to_results, f)

In [7]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Rewrite/Results/case2_middle_to_results.txt", "rb") as f:
    case2_middle_to_results = pickle.load(f)

In [106]:
ids_to_move = [16]

case2_loc_results = case2_rewrite(pairs_to_observe, ids_to_move)

with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Rewrite/Results/case2_loc_results.txt", "wb") as f:
    pickle.dump(case2_loc_results, f)

In [107]:
ids_to_move = [12, 13, 14, 15]

case2_allexceptloc_results = case2_rewrite(pairs_to_observe, ids_to_move)

with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Rewrite/Results/case2_allexceptloc_results.txt", "wb") as f:
    pickle.dump(case2_allexceptloc_results, f)

In [108]:
ids_to_move = [12, 13, 14, 15, 16]

case2_alltok_results = case2_rewrite(pairs_to_observe, ids_to_move)

with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Rewrite/Results/case2_alltok_results.txt", "wb") as f:
    pickle.dump(case2_alltok_results, f)

In [119]:
ids_to_move = list(range(16))

case2_all3sentexceptlastloc_results = case2_rewrite(pairs_to_observe, ids_to_move)

with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Rewrite/Results/case2_all3sentexceptlastloc_results.txt", "wb") as f:
    pickle.dump(case2_all3sentexceptlastloc_results, f)

# Result Inspection

In [4]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Rewrite/Results/case1_middle_to_results.txt", "rb") as f:
    case1_middle_to_results = pickle.load(f)
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Rewrite/Results/case1_loc_results.txt", "rb") as f:
    case1_loc_results = pickle.load(f)
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Rewrite/Results/case1_allexceptloc_results.txt", "rb") as f:
    case1_allexceptloc_results = pickle.load(f)
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Rewrite/Results/case1_alltok_results.txt", "rb") as f:
    case1_alltok_results = pickle.load(f)
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Rewrite/Results/case1_all3sentexceptlastloc_results.txt", "rb") as f:
    case1_all3sentexceptlastloc_results = pickle.load(f)
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Rewrite/Results/case2_middle_to_results.txt", "rb") as f:
    case2_middle_to_results = pickle.load(f)
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Rewrite/Results/case2_loc_results.txt", "rb") as f:
    case2_loc_results = pickle.load(f)
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Rewrite/Results/case2_allexceptloc_results.txt", "rb") as f:
    case2_allexceptloc_results = pickle.load(f)
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Rewrite/Results/case2_alltok_results.txt", "rb") as f:
    case2_alltok_results = pickle.load(f)
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Rewrite/Results/case2_all3sentexceptlastloc_results.txt", "rb") as f:
    case2_all3sentexceptlastloc_results = pickle.load(f)

In [6]:
def get_token_probability(model_out, token):
    global next_token_scores
    token_id = tokenizer.encode(token)[0]
    next_token_scores = torch.nn.functional.softmax(model_out.scores[0].float(), dim=-1)  # (batch_size * num_beams, vocab_size)
    probability = next_token_scores[0][token_id].item()
    
    _, indices = torch.sort(next_token_scores, descending=True)
    max_prob_ind = torch.where(indices[0] == token_id)[0].item() + 1
    
    return probability, max_prob_ind



In [22]:
def check_rewrite_results(results):

    sentence_1_acc = []
    sentence_2_acc = []

    sentence_rewritten_answer_1 = []
    sentence_rewritten_answer_2 = []

    generation_lens = defaultdict(int)

    probability_results = np.zeros((3, 2, 3**6))
    probability_results_diff = np.zeros((3, 2, 3**6))
    sorted_results = np.zeros((3, 2, 3**6))
    sorted_results_diff = np.zeros((3, 2, 3**6))

    for i, result in enumerate(results):
        context_1 = result.context_1    
        context_2 = result.context_2
        generation_1 = result.generation_1
        generation_2 = result.generation_2
        generation_rewritten = result.generation_rewritten
        answer_1 = result.answer_1
        answer_2 = result.answer_2

        out_sequence_1 = tokenizer.decode(generation_1.sequences[0], skip_special_tokens=True)
        out_sequence_2 = tokenizer.decode(generation_2.sequences[0], skip_special_tokens=True)
        out_sequence_rewritten = tokenizer.decode(generation_rewritten.sequences[0], skip_special_tokens=True)

        gen_1_ans_1, ind_11 = get_token_probability(generation_1, answer_1)
        gen_1_ans_2, ind_12 = get_token_probability(generation_1, answer_2)    
        gen_2_ans_1, ind_21 = get_token_probability(generation_2, answer_1)
        gen_2_ans_2, ind_22 = get_token_probability(generation_2, answer_2)
        gen_rw_ans_1, ind_rw1 = get_token_probability(generation_rewritten, answer_1)
        gen_rw_ans_2, ind_rw2 = get_token_probability(generation_rewritten, answer_2)

        """print(gen_1_ans_1)
        print(gen_1_ans_2)
        print(gen_2_ans_1)
        print(gen_2_ans_2)
        print(gen_rw_ans_1)
        print(gen_rw_ans_2)

        print(ind_11)
        print(ind_12)
        print(ind_21)
        print(ind_22)
        print(ind_rw1)
        print(ind_rw2)"""

        generation_lens[len(generation_1.sequences[0])] += 1
        generation_lens[len(generation_2.sequences[0])] += 1
        generation_lens[len(generation_rewritten.sequences[0])] += 1

        if answer_1 in out_sequence_1:
            sentence_1_acc.append(1)
        else:
            sentence_1_acc.append(0)
        if answer_2 in out_sequence_2:
            sentence_2_acc.append(1)
        else:
            sentence_2_acc.append(0)      

        if answer_1 in out_sequence_rewritten:
            sentence_rewritten_answer_1.append(1)
        else:
            sentence_rewritten_answer_1.append(0)
        if answer_2 in out_sequence_rewritten:
            sentence_rewritten_answer_2.append(1)
        else:
            sentence_rewritten_answer_2.append(0)
            
        probability_results[:, :, i] = [ [gen_1_ans_1, gen_1_ans_2],
                                         [gen_2_ans_1, gen_2_ans_2], 
                                         [gen_rw_ans_1 , gen_rw_ans_2] ]

        probability_results_diff[:, :, i] = [ [gen_1_ans_1 - gen_2_ans_1, gen_1_ans_2 - gen_2_ans_2],
                                         [gen_rw_ans_1 - gen_1_ans_1, gen_rw_ans_2 - gen_1_ans_2], 
                                         [gen_rw_ans_1 - gen_2_ans_1, gen_rw_ans_2 - gen_2_ans_2]]

        sorted_results[:, :, i] = [  [ind_11, ind_12],
                                     [ind_21, ind_22], 
                                     [ind_rw1, ind_rw2]]

        sorted_results_diff[:, :, i] = [  [ind_11 - ind_21, ind_12 - ind_22],
                                     [ind_rw1 - ind_11, ind_rw2 - ind_12], 
                                     [ind_rw1 - ind_21, ind_rw2 - ind_22]]

    acc_1 = sum(sentence_1_acc)/len(sentence_1_acc)
    acc_2 = sum(sentence_2_acc)/len(sentence_2_acc)

    acc_rw1 = sum(sentence_rewritten_answer_1)/len(sentence_rewritten_answer_1)
    acc_rw2 = sum(sentence_rewritten_answer_2)/len(sentence_rewritten_answer_2)
    
    print("Accuracy of context 1, city1:", acc_1)
    print("Accuracy of context 2, city2:", acc_2)
    print("Accuracy of rewritten context, city 1:", acc_rw1)
    print("Accuracy of rewritten context, city 2:", acc_rw2)
    print()
    print("Probability of generating the answer tokens: ", np.average(probability_results, axis=-1).tolist())
    print("Probability of generating the answer tokens, difference: ", np.average(probability_results_diff, axis=-1).tolist())
    print("Token generation index:", np.average(sorted_results, axis=-1).tolist())
    print("Token generation index, difference:", np.average(sorted_results_diff, axis=-1).tolist())
    
    return acc_1, acc_2, acc_rw1, acc_rw2, generation_lens, probability_results, probability_results_diff, sorted_results, sorted_results_diff

## Case 1

In [23]:
acc_1, acc_2, acc_rw1, acc_rw2, generation_lens, probability_results, probability_results_diff, sorted_results, sorted_results_diff = check_rewrite_results(case1_middle_to_results)

Accuracy of context 1, city1: 1.0
Accuracy of context 2, city2: 1.0
Accuracy of rewritten context, city 1: 0.0
Accuracy of rewritten context, city 2: 1.0

Probability of generating the answer tokens:  [[0.9578431850434658, 3.6188950995433694e-05], [1.0024356529266287e-05, 0.9808337654923542], [9.977434288914297e-06, 0.9809350096639783]]
Probability of generating the answer tokens, difference:  [[0.9578331606869366, -0.9807975765413588], [-0.9578332076091768, 0.9808988207129828], [-4.692224035198956e-08, 0.00010124417162399397]]
Token generation index: [[1.0, 269.997256515775], [192.77366255144034, 1.0], [192.31001371742113, 1.0]]
Token generation index, difference: [[-191.77366255144034, 268.997256515775], [191.31001371742113, -268.997256515775], [-0.46364883401920437, 0.0]]


In [24]:
acc_1, acc_2, acc_rw1, acc_rw2, generation_lens, probability_results, probability_results_diff, sorted_results, sorted_results_diff = check_rewrite_results(case1_loc_results)

Accuracy of context 1, city1: 1.0
Accuracy of context 2, city2: 1.0
Accuracy of rewritten context, city 1: 1.0
Accuracy of rewritten context, city 2: 0.0

Probability of generating the answer tokens:  [[0.9578431850434658, 3.6188950995433694e-05], [1.0024356529266287e-05, 0.9808337654923542], [0.9565908590789031, 3.736941194759639e-05]]
Probability of generating the answer tokens, difference:  [[0.9578331606869366, -0.9807975765413588], [-0.001252325964562687, 1.1804609521626924e-06], [0.9565808347223739, -0.9807963960804067]]
Token generation index: [[1.0, 269.997256515775], [192.77366255144034, 1.0], [1.0, 272.7818930041152]]
Token generation index, difference: [[-191.77366255144034, 268.997256515775], [0.0, 2.784636488340192], [-191.77366255144034, 271.7818930041152]]


In [25]:
acc_1, acc_2, acc_rw1, acc_rw2, generation_lens, probability_results, probability_results_diff, sorted_results, sorted_results_diff = check_rewrite_results(case1_allexceptloc_results)

Accuracy of context 1, city1: 1.0
Accuracy of context 2, city2: 1.0
Accuracy of rewritten context, city 1: 0.0
Accuracy of rewritten context, city 2: 1.0

Probability of generating the answer tokens:  [[0.9578431850434658, 3.6188950995433694e-05], [1.0024356529266287e-05, 0.9808337654923542], [1.005252838808857e-05, 0.9801506396837849]]
Probability of generating the answer tokens, difference:  [[0.9578331606869366, -0.9807975765413588], [-0.9578331325150777, 0.9801144507327895], [2.817185882228476e-08, -0.0006831258085693025]]
Token generation index: [[1.0, 269.997256515775], [192.77366255144034, 1.0], [191.93141289437585, 1.0]]
Token generation index, difference: [[-191.77366255144034, 268.997256515775], [190.93141289437585, -268.997256515775], [-0.8422496570644719, 0.0]]


In [26]:
acc_1, acc_2, acc_rw1, acc_rw2, generation_lens, probability_results, probability_results_diff, sorted_results, sorted_results_diff = check_rewrite_results(case1_alltok_results)

Accuracy of context 1, city1: 1.0
Accuracy of context 2, city2: 1.0
Accuracy of rewritten context, city 1: 1.0
Accuracy of rewritten context, city 2: 0.0

Probability of generating the answer tokens:  [[0.9578431850434658, 3.6188950995433694e-05], [1.0024356529266287e-05, 0.9808337654923542], [0.9570660140615759, 3.609393726884016e-05]]
Probability of generating the answer tokens, difference:  [[0.9578331606869366, -0.9807975765413588], [-0.0007771709818898896, -9.501372659354153e-08], [0.9570559897050467, -0.9807976715550853]]
Token generation index: [[1.0, 269.997256515775], [192.77366255144034, 1.0], [1.0, 274.91495198902606]]
Token generation index, difference: [[-191.77366255144034, 268.997256515775], [0.0, 4.917695473251029], [-191.77366255144034, 273.91495198902606]]


In [27]:
acc_1, acc_2, acc_rw1, acc_rw2, generation_lens, probability_results, probability_results_diff, sorted_results, sorted_results_diff = check_rewrite_results(case1_all3sentexceptlastloc_results)

Accuracy of context 1, city1: 1.0
Accuracy of context 2, city2: 1.0
Accuracy of rewritten context, city 1: 0.0
Accuracy of rewritten context, city 2: 1.0

Probability of generating the answer tokens:  [[0.9578431850434658, 3.6188950995433694e-05], [1.0024356529266287e-05, 0.9808337654923542], [9.341803144641105e-06, 0.980047610725068]]
Probability of generating the answer tokens, difference:  [[0.9578331606869366, -0.9807975765413588], [-0.9578338432403212, 0.9800114217740725], [-6.825533846251811e-07, -0.0007861547672863032]]
Token generation index: [[1.0, 269.997256515775], [192.77366255144034, 1.0], [198.22085048010973, 1.0]]
Token generation index, difference: [[-191.77366255144034, 268.997256515775], [197.22085048010973, -268.997256515775], [5.44718792866941, 0.0]]


## Case 2

In [28]:
acc_1, acc_2, acc_rw1, acc_rw2, generation_lens, probability_results, probability_results_diff, sorted_results, sorted_results_diff = check_rewrite_results(case2_middle_to_results)

Accuracy of context 1, city1: 1.0
Accuracy of context 2, city2: 1.0
Accuracy of rewritten context, city 1: 0.0
Accuracy of rewritten context, city 2: 1.0

Probability of generating the answer tokens:  [[0.9578431850434658, 3.6188950995433694e-05], [1.0024356529266287e-05, 0.9808337654923542], [1.0093023461045406e-05, 0.9807449892388123]]
Probability of generating the answer tokens, difference:  [[0.9578331606869366, -0.9807975765413588], [-0.9578330920200048, 0.9807088002878168], [6.866693177911862e-08, -8.877625354196473e-05]]
Token generation index: [[1.0, 269.997256515775], [192.77366255144034, 1.0], [193.90946502057614, 1.0]]
Token generation index, difference: [[-191.77366255144034, 268.997256515775], [192.90946502057614, -268.997256515775], [1.1358024691358024, 0.0]]


In [29]:
acc_1, acc_2, acc_rw1, acc_rw2, generation_lens, probability_results, probability_results_diff, sorted_results, sorted_results_diff = check_rewrite_results(case2_loc_results)

Accuracy of context 1, city1: 1.0
Accuracy of context 2, city2: 1.0
Accuracy of rewritten context, city 1: 1.0
Accuracy of rewritten context, city 2: 0.0

Probability of generating the answer tokens:  [[0.9578431850434658, 3.6188950995433694e-05], [1.0024356529266287e-05, 0.9808337654923542], [0.9593255508420235, 3.8994317712524644e-05]]
Probability of generating the answer tokens, difference:  [[0.9578331606869366, -0.9807975765413588], [0.0014823657985577367, 2.8053667170909503e-06], [0.9593155264854943, -0.9807947711746418]]
Token generation index: [[1.0, 269.997256515775], [192.77366255144034, 1.0], [1.0, 230.88477366255145]]
Token generation index, difference: [[-191.77366255144034, 268.997256515775], [0.0, -39.1124828532236], [-191.77366255144034, 229.88477366255145]]


In [30]:
acc_1, acc_2, acc_rw1, acc_rw2, generation_lens, probability_results, probability_results_diff, sorted_results, sorted_results_diff = check_rewrite_results(case2_allexceptloc_results)

Accuracy of context 1, city1: 1.0
Accuracy of context 2, city2: 1.0
Accuracy of rewritten context, city 1: 0.0
Accuracy of rewritten context, city 2: 1.0

Probability of generating the answer tokens:  [[0.9578431850434658, 3.6188950995433694e-05], [1.0024356529266287e-05, 0.9808337654923542], [1.0353784404255143e-05, 0.9796839546110732]]
Probability of generating the answer tokens, difference:  [[0.9578331606869366, -0.9807975765413588], [-0.9578328312590615, 0.9796477656600777], [3.294278749888563e-07, -0.0011498108812810952]]
Token generation index: [[1.0, 269.997256515775], [192.77366255144034, 1.0], [186.86008230452674, 1.0]]
Token generation index, difference: [[-191.77366255144034, 268.997256515775], [185.86008230452674, -268.997256515775], [-5.91358024691358, 0.0]]


In [31]:
acc_1, acc_2, acc_rw1, acc_rw2, generation_lens, probability_results, probability_results_diff, sorted_results, sorted_results_diff = check_rewrite_results(case2_alltok_results)

Accuracy of context 1, city1: 1.0
Accuracy of context 2, city2: 1.0
Accuracy of rewritten context, city 1: 1.0
Accuracy of rewritten context, city 2: 0.0

Probability of generating the answer tokens:  [[0.9578431850434658, 3.6188950995433694e-05], [1.0024356529266287e-05, 0.9808337654923542], [0.9574766288241895, 3.673426079406445e-05]]
Probability of generating the answer tokens, difference:  [[0.9578331606869366, -0.9807975765413588], [-0.0003665562192762503, 5.453097986307557e-07], [0.9574666044676603, -0.9807970312315601]]
Token generation index: [[1.0, 269.997256515775], [192.77366255144034, 1.0], [1.0, 262.22770919067216]]
Token generation index, difference: [[-191.77366255144034, 268.997256515775], [0.0, -7.769547325102881], [-191.77366255144034, 261.22770919067216]]


In [32]:
acc_1, acc_2, acc_rw1, acc_rw2, generation_lens, probability_results, probability_results_diff, sorted_results, sorted_results_diff = check_rewrite_results(case2_all3sentexceptlastloc_results)

Accuracy of context 1, city1: 1.0
Accuracy of context 2, city2: 1.0
Accuracy of rewritten context, city 1: 0.0
Accuracy of rewritten context, city 2: 1.0

Probability of generating the answer tokens:  [[0.9578431850434658, 3.6188950995433694e-05], [1.0024356529266287e-05, 0.9808337654923542], [9.973773307080501e-06, 0.9791468326283417]]
Probability of generating the answer tokens, difference:  [[0.9578331606869366, -0.9807975765413588], [-0.9578332112701587, 0.9791106436773462], [-5.058322218578618e-08, -0.0016869328640125416]]
Token generation index: [[1.0, 269.997256515775], [192.77366255144034, 1.0], [196.14677640603566, 1.0]]
Token generation index, difference: [[-191.77366255144034, 268.997256515775], [195.14677640603566, -268.997256515775], [3.373113854595336, 0.0]]


In [15]:
import sys
sys.path.append('/kuacc/users/bozyurt20/.conda/envs/hf/lib/python3.8/site-packages/transformers/models/')
cp modeling_t5.py /kuacc/users/bozyurt20/.conda/envs/hf/lib/python3.8/site-packages/transformers/models/t5/
cp utils.py /kuacc/users/bozyurt20/.conda/envs/hf/lib/python3.8/site-packages/transformers/generation/
cp modeling_outputs.py /kuacc/users/bozyurt20/.conda/envs/hf/lib/python3.8/site-packages/transformers/

SyntaxError: invalid syntax (676971914.py, line 3)